<a href="https://colab.research.google.com/github/sanadv/SkinCancerClassificationModels/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (Layer, Input, Lambda, Conv2D, MaxPooling2D, Conv2DTranspose,
                                     BatchNormalization, LeakyReLU, Dropout, concatenate,
                                     Average, Resizing, multiply, Flatten, Dense)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import MultiHeadAttention
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Concatenate, Average, GlobalAveragePooling2D, Dense, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
#from vit_keras import vit
import numpy as np
from tensorflow.keras.layers import BatchNormalization, Conv2D, LeakyReLU, MaxPooling2D
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, concatenate, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall, AUC
from sklearn.metrics import f1_score
from tensorflow.keras.layers import multiply
from tensorflow.keras.layers import Lambda
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Resizing
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda, Average
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import GaussianNoise

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten,Lambda
from tensorflow.keras.layers import Average, Lambda, Conv2D, Resizing, UpSampling2D, MaxPooling2D
import time
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.layers import Dense, Dropout, Concatenate, Flatten,Reshape

# Define the ensemble model creation function
from tensorflow.keras.layers import Conv2D, LayerNormalization, MultiHeadAttention

from tensorflow.keras.callbacks import ReduceLROnPlateau


from tensorflow.keras.applications import Xception, InceptionV3, NASNetMobile
from tensorflow.keras.layers import DepthwiseConv2D, Add, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten,BatchNormalization,Activation
from tensorflow.keras.regularizers import l2
from vit_keras import vit
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply,ReLU
from keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Add, Reshape, multiply
from keras.regularizers import l2
from keras.models import Model
from keras.layers import Layer, MultiHeadAttention

from google.colab import drive
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Reshape, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras import layers


drive.mount('/content/drive')


extract_path = '/content/drive/MyDrive/hamdataset'

# Parameters
batch_size = 8
epochs = 50
learning_rate = 1e-4

# Define the input 3 and batch size
input_shape = (128, 128, 3)  # 128x128 with RGB images (3 channels)

# Initialize the ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training data generator
train_generator = datagen.flow_from_directory(
    extract_path,  # Replace with your directory path
    target_size=input_shape[:2],  # Resize images to 128x128
    color_mode='rgb',  # Set color mode to RGB
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

# Validation data generator
validation_generator = datagen.flow_from_directory(
    extract_path,  # Replace with your directory path
    target_size=input_shape[:2],  # Resize images to 128x128
    color_mode='rgb',  # Set color mode to RGB
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras import layers





def squeeze_excite_block(input, ratio=8):
    init = input
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)

    x = multiply([init, se])
    return x

class SelfAttentionLayer(Layer):
    def __init__(self, embed_dim=32, num_heads=2, **kwargs):
        super(SelfAttentionLayer, self).__init__(**kwargs)
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

    def call(self, inputs):
        return self.attention(inputs, inputs)

def depthwise_separable_conv_block(inputs, num_filters, kernel_size=3, padding='same'):
    x = DepthwiseConv2D(kernel_size=kernel_size, padding=padding)(inputs)
    x = Conv2D(num_filters, kernel_size=1, padding=padding)(x)
    return x

def build_vit_model_with_transfer_learningOP2(input_shape, num_classes=3):
    input_layer = Input(input_shape)

    def edge_detection_layer(x):
        sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
        sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float32)
        sobel_x = np.expand_dims(sobel_x, axis=-1)
        sobel_x = np.repeat(sobel_x, 3, axis=-1)
        sobel_x = np.expand_dims(sobel_x, axis=-1)
        sobel_y = np.expand_dims(sobel_y, axis=-1)
        sobel_y = np.repeat(sobel_y, 3, axis=-1)
        sobel_y = np.expand_dims(sobel_y, axis=-1)
        sobel_x = tf.convert_to_tensor(sobel_x)
        sobel_y = tf.convert_to_tensor(sobel_y)
        sobel_x = tf.nn.depthwise_conv2d(x, sobel_x, strides=[1, 1, 1, 1], padding='SAME')
        sobel_y = tf.nn.depthwise_conv2d(x, sobel_y, strides=[1, 1, 1, 1], padding='SAME')
        edges = tf.math.abs(sobel_x) + tf.math.abs(sobel_y)
        return edges

    def orientation_pooling(inputs):
        rotations = [0, 90, 180, 270]
        outputs = []
        for angle in rotations:
            rotated = Lambda(lambda x: tf.image.rot90(x, k=angle // 90))(inputs)
            outputs.append(rotated)
        return Average()(outputs)

    edges = Lambda(edge_detection_layer)(input_layer)

    # U-Net with reduced filters
    c1 = depthwise_separable_conv_block(input_layer, 32)
    c1 = Dropout(0.1)(c1)
    c1 = depthwise_separable_conv_block(c1, 32)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = depthwise_separable_conv_block(p1, 64)
    c2 = Dropout(0.1)(c2)
    c2 = depthwise_separable_conv_block(c2, 64)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = depthwise_separable_conv_block(p2, 128)
    c3 = Dropout(0.2)(c3)
    c3 = depthwise_separable_conv_block(c3, 128)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = depthwise_separable_conv_block(p3, 256)
    c4 = Dropout(0.2)(c4)
    c4 = depthwise_separable_conv_block(c4, 256)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = depthwise_separable_conv_block(p4, 512)
    c5 = Dropout(0.3)(c5)
    c5 = depthwise_separable_conv_block(c5, 512)

    u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = depthwise_separable_conv_block(u6, 256)
    c6 = Dropout(0.2)(c6)
    c6 = depthwise_separable_conv_block(c6, 256)

    u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = depthwise_separable_conv_block(u7, 128)
    c7 = Dropout(0.2)(c7)
    c7 = depthwise_separable_conv_block(c7, 128)

    u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = depthwise_separable_conv_block(u8, 64)
    c8 = Dropout(0.1)(c8)
    c8 = depthwise_separable_conv_block(c8, 64)

    u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = depthwise_separable_conv_block(u9, 32)
    c9 = Dropout(0.1)(c9)
    c9 = depthwise_separable_conv_block(c9, 32)

    x = orientation_pooling(c9)
    x = Conv2D(3, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same')(x)

    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
        # Continue from Dropout
    x = Dropout(0.4)(x)
    target_height, target_width = x.shape[1], x.shape[2]
    resized_edges = Resizing(target_height, target_width)(edges)

    # Concatenate x and resized edges
    x = concatenate([x, resized_edges])
    x = orientation_pooling(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.4)(x)
    x = squeeze_excite_block(x)
    x = SelfAttentionLayer(embed_dim=32, num_heads=2)(x)
    x = Conv2D(3, (1, 1), activation='relu', kernel_initializer='he_normal', padding='same')(x)


    vit_model = vit.vit_b32(
        image_size=input_shape[0],
        activation='softmax',
        pretrained=True,
        include_top=True,
        pretrained_top=False,
        classes=num_classes
    )(x)

    return Model(inputs=input_layer, outputs=vit_model)



def squeeze_excite_block(input, ratio=16):
    init = input
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
 # Normalization
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    print("Shape after LayerNormalization:", x.shape)  # Debugging print

    # Expanding the dimensions to add sequence length
    x = tf.expand_dims(x, axis=1)
    print("Shape after expanding dimensions:", x.shape)  # Debugging print

    # Apply MultiHeadAttention
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    print("Shape after MultiHeadAttention:", x.shape)  # Debugging print

    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)
    return x + res

def build_cnn_with_attention(input_shape, num_classes, embed_dim=32, num_heads=2, num_transformer_layers=2, ff_dim=32):
    inputs = Input(input_shape)

    # DenseNet121 for Feature Extraction
    densenet = DenseNet121(include_top=False, input_tensor=inputs, weights='imagenet')
    densenet.trainable = False

    # Custom layers onto densenet
    x = densenet.output
    x = Conv2D(16, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = squeeze_excite_block(x)

    x = Conv2D(16, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = squeeze_excite_block(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    # Flatten and apply Transformer blocks
    x = Flatten()(x)
    x = Dense(embed_dim)(x)  # Projecting to transformer dimension

    for _ in range(num_transformer_layers):
        x = transformer_encoder(x, embed_dim, num_heads, ff_dim)

    # Classifier head
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)

    print("Shape before softmax:", x.shape)  # Add this line for debugging

    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model


from tensorflow.keras.applications.efficientnet import EfficientNetB0


def build_densnet(input_shape, num_classes,embed_dim=32):
    inputs = Input(input_shape)

    # DenseNet121 for Feature Extraction
    densenet = DenseNet121(include_top=False, input_tensor=inputs, weights='imagenet')
    densenet.trainable = False

    # Custom layers onto densenet
    x = densenet.output

    # Adding layers to process the output into a suitable form for classification
    x = GlobalAveragePooling2D()(x)
    x = Dense(embed_dim, activation='relu')(x)  # Embedding layer (optional, can adjust the size)
    x = Dense(num_classes, activation='softmax')(x)  # Output layer for classification

    model = Model(inputs, x)
    return model

def build_efficientnet_model(input_shape, num_classes):
    # Load the pre-trained EfficientNetB0 model without the top layer
    base_model = EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')

    # Freeze the base_model
    base_model.trainable = False

    # Create the input layer
    inputs = Input(shape=input_shape)

    # Use the EfficientNetB0 model as the base
    x = base_model(inputs, training=False)  # Make sure the base model is running in inference mode here

    # Add GlobalAveragePooling2D layer to reduce feature maps size
    x = GlobalAveragePooling2D()(x)

    # Add a Dense layer with a softmax activation for the classification
    outputs = Dense(num_classes, activation='softmax')(x)

    # Construct the final model
    model = Model(inputs, outputs)

    return model
def build_vit_model(input_shape, num_classes):
    input_layer = Input(input_shape)
    vit_model = vit.vit_b32(
        image_size=input_shape[0],
        activation='softmax',
        pretrained=True,
        include_top=True,
        pretrained_top=False,
        classes=num_classes
    )(input_layer)

    return Model(inputs=input_layer, outputs=vit_model)

def build_mobilenet_model(input_shape, num_classes=3):
    input_layer = Input(shape=input_shape)

    # Use MobileNetV2 directly on the input_layer
    mobilenet_model = tf.keras.applications.MobileNetV2(input_shape=input_shape[:2] + (3,), include_top=False)(input_layer)

    x = GlobalAveragePooling2D()(mobilenet_model)
    output_layer = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=input_layer, outputs=output_layer)

def build_resnet_model(input_shape, num_classes=3):
    input_layer = Input(shape=input_shape)
    resnet_model = tf.keras.applications.ResNet50(input_shape=input_shape[:2] + (3,), include_top=False)(input_layer)
    x = GlobalAveragePooling2D()(resnet_model)
    output_layer = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=input_layer, outputs=output_layer)


def build_unet_model(input_shape,num_classes):
    unet_input = Input(shape=input_shape)

    # Contracting Path
    c1 = Conv2D(32, (3,3), padding='same', activation='relu')(unet_input)
    p1 = MaxPooling2D((2,2))(c1)
    c2 = Conv2D(64, (3,3), padding='same', activation='relu')(p1)
    p2 = MaxPooling2D((2,2))(c2)
    c3 = Conv2D(128, (3,3), padding='same', activation='relu')(p2)
    p3 = MaxPooling2D((2,2))(c3)
    c4 = Conv2D(256, (3,3), padding='same', activation='relu')(p3)

    # Expansive Path
    u1 = UpSampling2D((2,2))(c4)
    m1 = concatenate([u1, c3])
    c5 = Conv2D(128, (3,3), padding='same', activation='relu')(m1)
    u2 = UpSampling2D((2,2))(c5)
    m2 = concatenate([u2, c2])
    c6 = Conv2D(64, (3,3), padding='same', activation='relu')(m2)
    u3 = UpSampling2D((2,2))(c6)
    m3 = concatenate([u3, c1])
    c7 = Conv2D(32, (3,3), padding='same', activation='relu')(m3)

    # Final Layers
    gap = GlobalAveragePooling2D()(c7)
    unet_output = Dense(num_classes, activation='softmax')(gap)

    return Model(unet_input, unet_output)

# ... [previous imports remain unchanged]

def build_darknet_model(input_shape, num_classes):
    # Simplified DarkNet-19 model
    def conv_block(x, filters, kernel_size, strides=(1, 1), max_pool=True):
        x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.1)(x)
        if max_pool:
            x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)
        return x

    input_layer = Input(shape=input_shape)

    x = conv_block(input_layer, 32, (3, 3))
    x = conv_block(x, 64, (3, 3))
    x = conv_block(x, 128, (3, 3), max_pool=False)
    x = conv_block(x, 64, (1, 1), max_pool=False)
    x = conv_block(x, 128, (3, 3))
    x = conv_block(x, 256, (3, 3), max_pool=False)
    x = conv_block(x, 128, (1, 1), max_pool=False)
    x = conv_block(x, 256, (3, 3))
    x = conv_block(x, 512, (3, 3), max_pool=False)
    x = conv_block(x, 256, (1, 1), max_pool=False)
    x = conv_block(x, 512, (3, 3), max_pool=False)
    x = conv_block(x, 256, (1, 1), max_pool=False)
    x = conv_block(x, 512, (3, 3))
    x = conv_block(x, 1024, (3, 3), max_pool=False)
    x = conv_block(x, 512, (1, 1), max_pool=False)
    x = conv_block(x, 1024, (3, 3), max_pool=False)
    x = conv_block(x, 512, (1, 1), max_pool=False)
    x = conv_block(x, 1024, (3, 3))

    x = GlobalAveragePooling2D()(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=input_layer, outputs=output_layer)

def fire_module(x, fire_id, squeeze=16, expand=64):
    f_name = "fire" + str(fire_id) + "/"

    # Squeeze layer
    x = Conv2D(squeeze, (1, 1), padding='valid', name=f_name + "squeeze1x1")(x)
    x = Activation('relu', name=f_name + "squeeze1x1_activation")(x)

    # Expand layer 1x1
    left = Conv2D(expand, (1, 1), padding='valid', name=f_name + "expand1x1")(x)
    left = Activation('relu', name=f_name + "expand1x1_activation")(left)

    # Expand layer 3x3
    right = Conv2D(expand, (3, 3), padding='same', name=f_name + "expand3x3")(x)
    right = Activation('relu', name=f_name + "expand3x3_activation")(right)

    # Concatenate expand layers
    x = concatenate([left, right], axis=3, name=f_name + "concat")
    return x

def build_squeezenet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Initial convolution layer
    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same', name='conv1')(inputs)
    x = Activation('relu', name='conv1_activation')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1')(x)

    # Fire modules
    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool2')(x)
    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool3')(x)
    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)

    # Final layers
    x = Dropout(0.5, name='drop9')(x)
    x = Conv2D(num_classes, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='conv10_activation')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='softmax')(x)

    model = Model(inputs, outputs=x)
    return model




def squeeze_excite_block(input, ratio=16):
    init = input
    channel_axis = -1  # assuming channels last format
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)

    x = multiply([init, se])
    return x
class SelfAttentionLayer(Layer):
    def __init__(self, embed_dim, num_heads, **kwargs):
        super(SelfAttentionLayer, self).__init__(**kwargs)
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

    def call(self, inputs):
        # Self attention
        return self.attention(inputs, inputs)

def build_cnn_with_attentionNodens(input_shape, num_classes, embed_dim=32, num_heads=2):
    input_layer = Input(input_shape)

    # Layer 1
    x = Conv2D(16, (3, 3), padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = squeeze_excite_block(x)

    # Layer 2
    x = Conv2D(16, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)
    x = SelfAttentionLayer(embed_dim, num_heads)(x)

    # Layer 3
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = squeeze_excite_block(x)

    # Layer 4
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Layer 5
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)
    x = SelfAttentionLayer(embed_dim, num_heads)(x)

    # Fully Connected Layers
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model


from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply
import tensorflow as tf

def ChannelAttentionModule(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = tf.keras.layers.GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = tf.keras.layers.Add()([avg_pool, max_pool])
    cbam_feature = tf.keras.activations.sigmoid(cbam_feature)

    return Multiply()([input_feature, cbam_feature])
from tensorflow.keras.layers import Conv2D, Concatenate, Activation

def SpatialAttentionModule(input_feature):
    kernel_size = 7

    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_feature)
    max_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_feature)
    concat = Concatenate(axis=-1)([avg_pool, max_pool])
    cbam_feature = Conv2D(filters=1,
                          kernel_size=kernel_size,
                          strides=1,
                          padding='same',
                          activation='sigmoid',
                          kernel_initializer='he_normal',
                          use_bias=False)(concat)

    return Multiply()([input_feature, cbam_feature])
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Dropout, Concatenate
from tensorflow.keras.applications import ResNet50, MobileNetV3Small
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import EfficientNetB0

from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

class EdgeDetectionLayer(Layer):
    def __init__(self):
        super(EdgeDetectionLayer, self).__init__()
        # Sobel filters
        sobel_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32)
        sobel_y = tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=tf.float32)
        sobel_x = tf.reshape(sobel_x, [3, 3, 1, 1])
        sobel_y = tf.reshape(sobel_y, [3, 3, 1, 1])
        self.sobel_filters = tf.concat([sobel_x, sobel_y], axis=-1)

    def call(self, inputs):
        gray = tf.image.rgb_to_grayscale(inputs)
        filtered = tf.nn.conv2d(gray, self.sobel_filters, strides=[1, 1, 1, 1], padding='SAME')
        edges = tf.reduce_sum(tf.abs(filtered), axis=-1, keepdims=True)
        return edges
def build_cnn_with_attentionEdges(input_shape, num_classes, embed_dim=32, num_heads=2, num_transformer_layers=2, ff_dim=32):
    inputs = Input(input_shape)

    # Edge Detection Layer
    edge_layer = EdgeDetectionLayer()
    edges = edge_layer(inputs)

    # Replace one channel of the input with the edge map
    modified_input = tf.concat([inputs[..., :2], edges], axis=-1)

    # DenseNet121 for Feature Extraction with modified input
    densenet = DenseNet121(include_top=False, input_tensor=modified_input, weights='imagenet')
    densenet.trainable = False


    # Custom layers onto densenet
    x = densenet.output
    x = Conv2D(16, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = squeeze_excite_block(x)

    x = Conv2D(16, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = squeeze_excite_block(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    # Flatten and apply Transformer blocks
    x = Flatten()(x)
    x = Dense(embed_dim)(x)  # Projecting to transformer dimension

    for _ in range(num_transformer_layers):
        x = transformer_encoder(x, embed_dim, num_heads, ff_dim)

    # Classifier head
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)

    print("Shape before softmax:", x.shape)  # Add this line for debugging

    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model


def build_cnn_with_attentionandEdgesRes(input_shape, num_classes, embed_dim=32, num_heads=2, num_transformer_layers=2, ff_dim=32):

    inputs = Input(input_shape)

    # Apply Edge Detection
    edge_detector = EdgeDetectionLayer()
    edges = edge_detector(inputs)

    # Initialize DenseNet121
    densenet = DenseNet121(include_top=False, input_tensor=inputs, weights='imagenet', pooling=None)
    densenet.trainable = False

    # Select an intermediate layer from DenseNet121
    intermediate_layer = densenet.get_layer('conv5_block16_concat').output
    intermediate_layer_resized = Resizing(inputs.shape[1], inputs.shape[2])(intermediate_layer)

    # Concatenate edge features with intermediate DenseNet121 features
    concatenated = Concatenate()([intermediate_layer_resized, edges])

    # Additional custom layers
    x = Conv2D(64, (3, 3), padding='same')(concatenated)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)

    # Transformer blocks (assuming transformer_encoder is defined)
    x = Dense(embed_dim)(x)
    for _ in range(num_transformer_layers):
        x = transformer_encoder(x, embed_dim, num_heads, ff_dim)

    # Classifier head
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model
class laplacianGeometricHeatFlowLayer(Layer):
    def __init__(self, iterations=10, diffusion_rate=0.1, **kwargs):
        super(GeometricHeatFlowLayer, self).__init__(**kwargs)
        self.iterations = iterations
        self.diffusion_rate = diffusion_rate

    def call(self, inputs):
        x = inputs
        for _ in range(self.iterations):
            x = self.apply_diffusion(x)
        return x

    def apply_diffusion(self, x):
        # Laplacian kernel
        laplacian_kernel = tf.constant([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]], dtype=tf.float32)
        laplacian_kernel = tf.reshape(laplacian_kernel, [3, 3, 1, 1])
        laplacian_kernel = tf.tile(laplacian_kernel, [1, 1, tf.shape(x)[-1], 1])

        # Apply Laplacian operator
        laplacian = tf.nn.depthwise_conv2d(x, laplacian_kernel, strides=[1, 1, 1, 1], padding='SAME')

        # Update the image based on the diffusion equation
        return x + self.diffusion_rate * laplacian

class GeometricHeatFlowLayer(Layer):
    def __init__(self, iterations=10, diffusion_rate=0.1, **kwargs):
        super(GeometricHeatFlowLayer, self).__init__(**kwargs)
        self.iterations = iterations
        self.diffusion_rate = diffusion_rate

    def call(self, inputs):
        x = inputs
        for _ in range(self.iterations):
            x = self.apply_diffusion(x)
        return x

    def apply_diffusion(self, x):
        # Implement the diffusion process here
        # This is a simplified example; you might need a more complex diffusion model
        kernel = tf.constant([[0.05, 0.2, 0.05], [0.2, 0.2, 0.2], [0.05, 0.2, 0.05]], dtype=tf.float32)
        kernel = tf.reshape(kernel, [3, 3, 1, 1])
        kernel = tf.tile(kernel, [1, 1, tf.shape(x)[-1], 1])
        return x + self.diffusion_rate * tf.nn.depthwise_conv2d(x, kernel, strides=[1, 1, 1, 1], padding='SAME')


class AdvancedGeometricHeatFlowLayer(Layer):
    def __init__(self, iterations=10, **kwargs):
        super(AdvancedGeometricHeatFlowLayer, self).__init__(**kwargs)
        self.iterations = iterations
        self.diffusion_rate = 0.01

    def call(self, inputs):
        x = inputs
        for _ in range(self.iterations):
            x = self.apply_anisotropic_diffusion(x)
        return x

    def apply_anisotropic_diffusion(self, x):
        # Compute image gradients
        dx, dy = self.compute_gradients(x)

        # Compute edge-stopping function based on gradients
        c = tf.exp(-tf.sqrt(tf.square(dx) + tf.square(dy)))

        # Anisotropic diffusion
        x = x + self.diffusion_rate * self.divergence(c * dx, c * dy)
        return x

    def compute_gradients(self, x):
        # Sobel filters to approximate gradients
        sobel_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32, shape=[3, 3, 1, 1])
        sobel_y = tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=tf.float32, shape=[3, 3, 1, 1])

        sobel_x = tf.tile(sobel_x, [1, 1, tf.shape(x)[-1], 1])
        sobel_y = tf.tile(sobel_y, [1, 1, tf.shape(x)[-1], 1])

        dx = tf.nn.depthwise_conv2d(x, sobel_x, strides=[1, 1, 1, 1], padding='SAME')
        dy = tf.nn.depthwise_conv2d(x, sobel_y, strides=[1, 1, 1, 1], padding='SAME')

        return dx, dy
    def divergence(self, dx, dy):
        num_channels = tf.shape(dx)[-1]

        kernel_x = tf.constant([[-1, 1]], dtype=tf.float32)
        kernel_x = tf.reshape(kernel_x, [2, 1, 1, 1])
        kernel_x = tf.tile(kernel_x, [1, 1, num_channels, 1])

        kernel_y = tf.constant([[-1], [1]], dtype=tf.float32)
        kernel_y = tf.reshape(kernel_y, [1, 2, 1, 1])
        kernel_y = tf.tile(kernel_y, [1, 1, num_channels, 1])

        # Change padding from 'VALID' to 'SAME'
        div_x = tf.nn.depthwise_conv2d(dx, kernel_x, strides=[1, 1, 1, 1], padding='SAME')
        div_y = tf.nn.depthwise_conv2d(dy, kernel_y, strides=[1, 1, 1, 1], padding='SAME')

        return div_x + div_y

def build_cnn_with_attention_and_heat_flow(input_shape, num_classes, embed_dim=32, num_heads=2, num_transformer_layers=2, ff_dim=32):
    inputs = Input(input_shape)

    # Apply Geometric Heat Flow
    heat_flow_layer = AdvancedGeometricHeatFlowLayer()
    heat_flow = heat_flow_layer(inputs)

    # Rest of the model remains the same...
    # Initialize DenseNet121
    densenet = DenseNet121(include_top=False, input_tensor=inputs, weights='imagenet', pooling=None)
    densenet.trainable = False
    # Select an intermediate layer from DenseNet121
    intermediate_layer = densenet.get_layer('conv5_block16_concat').output
    intermediate_layer_resized = Resizing(inputs.shape[1], inputs.shape[2])(intermediate_layer)

    # Concatenate heat flow features with intermediate DenseNet121 features
    concatenated = Concatenate()([intermediate_layer_resized, heat_flow])

    # Additional custom layers
    x = Conv2D(64, (3, 3), padding='same')(concatenated)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)

    # Transformer blocks (assuming transformer_encoder is defined)
    x = Dense(embed_dim)(x)
    for _ in range(num_transformer_layers):
        x = transformer_encoder(x, embed_dim, num_heads, ff_dim)

    # Classifier head
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

def train_individual_model(model, train_data, val_data, epochs=100):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)

    start_time = time.time()

    history = model.fit(train_data, validation_data=val_data, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])

    end_time = time.time()

    print(f"Training time: {end_time - start_time:.2f} seconds")

    # Using history to print training metrics
    print("\nTraining Metrics:")
    print(f"Loss: {history.history['loss'][-1]}")
    print(f"Accuracy: {history.history['accuracy'][-1]}")
    print(f"Precision: {history.history['precision'][-1]}")
    print(f"Recall: {history.history['recall'][-1]}")
    f1 = 2 * (history.history['precision'][-1] * history.history['recall'][-1]) / (history.history['precision'][-1] + history.history['recall'][-1])
    print(f"F1 Score: {f1}")
    print(f"AUC: {history.history['auc'][-1]}")

    # Using history to print validation metrics
    print("\nValidation Metrics:")
    print(f"Loss: {history.history['val_loss'][-1]}")
    print(f"Accuracy: {history.history['val_accuracy'][-1]}")
    print(f"Precision: {history.history['val_precision'][-1]}")
    print(f"Recall: {history.history['val_recall'][-1]}")
    val_f1 = 2 * (history.history['val_precision'][-1] * history.history['val_recall'][-1]) / (history.history['val_precision'][-1] + history.history['val_recall'][-1])
    print(f"Val F1 Score: {val_f1}")
    print(f"Val AUC: {history.history['val_auc'][-1]}")

    return model
vit_model = train_individual_model(build_model(input_shape,7), train_generator, validation_generator)
